In [1]:
#ライブラリのインポート

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

In [10]:
#train_dataの読み込み
df_train = pd.read_csv('house_price_train.csv')

In [11]:
#説明変数、目的変数を作成

df_data0 = df_train.loc[:,['1stFlrSF', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtUnfSF','GarageArea',
                           'GrLivArea', 'LotArea', 'TotalBsmtSF', 'YearBuilt',
                           'GarageYrBlt', 'LotFrontage', 'MSSubClass', 'OpenPorchSF',
                           'TotRmsAbvGrd','YearRemodAdd', 'YrSold',
                           'OverallQual', 'GarageCars', 'FullBath', 'BsmtQual_Ex']]
df_target = df_train.loc[:,['SalePrice']]
df_add0 = pd.concat([df_data0, df_target], axis=1)

In [12]:
#df_data0について主成分分析

#標準化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
data0_std = std.fit_transform(df_data0)
target_std = std.fit_transform(df_target)

#主成分分析
from sklearn.decomposition import PCA
pca = PCA(n_components=len(df_data0.columns))
data0_pca = pca.fit_transform(data0_std)

In [13]:
#主成分のDataFrameを作成

list_col = ['pca_'+str(i) for i in range(len(df_data0.columns))]
df_data_pca = pd.DataFrame(data0_pca, columns=list_col)

In [14]:
#多項式回帰、正則化

#特徴量抽出
pca0 = df_data_pca.loc[:,['pca_0']].values
pca3 = df_data_pca.loc[:,['pca_3']].values
pca9 = df_data_pca.loc[:,['pca_9']].values

#変数変換
from sklearn.preprocessing import PolynomialFeatures
cubic = PolynomialFeatures(degree=3)

pca0_cubic = cubic.fit_transform(pca0)
pca3_cubic = cubic.fit_transform(pca3)
pca9_cubic = cubic.fit_transform(pca9)

#変数を連結,特徴量
pca_sel = np.hstack((pca0_cubic, pca3_cubic, pca9_cubic))

In [15]:
#モデルを学習、Lasso
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(pca_sel, df_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [18]:
df_test = pd.read_csv('house_price_test.csv')
X_test = df_test.loc[:,['1stFlrSF', 'BedroomAbvGr', 'BsmtFinSF1', 'BsmtUnfSF','GarageArea',
                        'GrLivArea', 'LotArea', 'TotalBsmtSF', 'YearBuilt',
                        'GarageYrBlt', 'LotFrontage', 'MSSubClass', 'OpenPorchSF',
                        'TotRmsAbvGrd','YearRemodAdd', 'YrSold',
                        'OverallQual', 'GarageCars', 'FullBath', 'BsmtQual_Ex']]
X_test_std = std.fit_transform(X_test)
pca_x = PCA(n_components=len(X_test.columns))
X_test_pca = pca_x.fit_transform(X_test_std)

In [19]:
list_x = ['pca_'+str(i) for i in range(len(X_test.columns))]
X_pca_df = pd.DataFrame(X_test_pca, columns=list_x)

In [21]:
#多項式回帰、正則化

#特徴量抽出
pca0 = X_pca_df.loc[:,['pca_0']].values
pca3 = X_pca_df.loc[:,['pca_3']].values
pca9 = X_pca_df.loc[:,['pca_9']].values

#変数変換
from sklearn.preprocessing import PolynomialFeatures
cubic = PolynomialFeatures(degree=3)

pca0_cubic = cubic.fit_transform(pca0)
pca3_cubic = cubic.fit_transform(pca3)
pca9_cubic = cubic.fit_transform(pca9)

#変数を連結,特徴量
X_fin = np.hstack((pca0_cubic, pca3_cubic, pca9_cubic))

In [24]:
#結果を出力
result = lr.predict(X_fin)

In [32]:
#csvファイルに変換

index_list = list(range(1461,2920,1))
result_ = pd.DataFrame(result,columns=['SalePrice'])
result_.index = index_list
result_.to_csv('result_1.csv')